It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
                                'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                    'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                    'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                    'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
                                '12. Air France', '"Swiss Air"']})

<h2>Problem Statement 1</h2>
Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [28]:
df = df.fillna(df.rolling(1).sum().shift()+10)
df['FlightNumber'] = df['FlightNumber'].astype(int)

<h2>Problem Statement 2</h2>
The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [4]:
df1 = pd.DataFrame(df.From_To.str.split('_',1).tolist(),columns=['From', 'To'])
df1

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


<h2>Problem Statement 3</h2>
Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [5]:
df1['From'] = df1.From.str.title()
df1['To'] = df1.To.str.title()

In [6]:
df1

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


<h2>Problem Statement 4</h2>
Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [7]:
df.drop('From_To', axis=1, inplace=True)

In [16]:
df = pd.concat([df1,df], axis=1)

In [17]:
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045.0,"[23, 47]",KLM(!)
1,Madrid,Milan,10055.0,[],<Air France> (12)
2,London,Stockholm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075.0,[13],12. Air France
4,Brussels,London,10085.0,"[67, 32]","""Swiss Air"""


<h2>Problem Statement 5</h2>
In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.

In [20]:
delay = pd.DataFrame(df['RecentDelays'].to_list(), columns= ['delay_1', 'delay_2', 'delay_3'])

In [22]:
df.drop('RecentDelays', axis=1, inplace=True)

In [24]:
df = pd.concat([df,delay], axis=1)

In [29]:
df

,From,To,FlightNumber,Airline,delay_1,delay_2,delay_3
0,London,Paris,10045,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055,<Air France> (12),33.0,57.0,NaN
2,London,Stockholm,10065,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075,12. Air France,13.0,53.0,97.0
4,Brussels,London,10085,"""Swiss Air""",67.0,32.0,107.0
